In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
from sklearn.utils import resample, shuffle

Have to swap axes for the convolution to work

In [2]:
# the data should be organized as [date, open, close, low, high, volume]
data = sio.loadmat('../daily_data.mat')['data']
blah = np.swapaxes(data, 1, 2)

In [3]:
data_len = data.shape[0]

# test data length
train_len = round(3 * data_len / 4)

x_train = data[0:train_len, :, :]
# start with just predicting the opening value the next day
y_train = data[1:train_len + 1, :, 1]

x_test = data[train_len + 1:-1, :, :]
# start with just predicting the opening value the next day
y_test = data[train_len + 2:, :, 1]

# let's start by just flattening the data
x_train = np.reshape(x_train, (train_len, -1))
x_test = np.reshape(x_test, (len(y_test), -1))

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [9]:
def conv1d(x, W):
    return tf.nn.conv1d(x, W, stride=6, padding='VALID')

Let's do the convolution layer

In [10]:
input_len = x_train.shape[1]
output_len = y_train.shape[1]
hidden_len = round((input_len-output_len)/2.0)

x = tf.placeholder(tf.float32, [None, input_len])
W_conv = weight_variable([6, 501, 501])
bias_conv = bias_variable([501])
x_conv = tf.nn.relu(conv1d(x, W_conv) + b_conv)

ValueError: Shape must be rank 4 but is rank 3 for 'conv1d/Conv2D' (op: 'Conv2D') with input shapes: [?,1,3006], [1,6,501,501].

In [ ]:

# Create the model
# x = tf.placeholder(tf.float32, [None, input_len])
# W = tf.Variable(tf.zeros([input_len, output_len]))
# b = tf.Variable(tf.zeros([output_len]))
# y = tf.matmul(x, W) + b
# x = tf.placeholder(tf.float32, [None, input_len])
W = weight_variable([input_len, hidden_len])
b = bias_variable([hidden_len])
x2 = tf.tanh(tf.matmul(x, W) + b)
# x2 = tf.nn.relu(tf.matmul(x, W) + b)

# add dropout
keep_prob = tf.placeholder(tf.float32)
x2_drop = tf.nn.dropout(x2, keep_prob)

W2 = weight_variable([hidden_len, output_len])
b2 = bias_variable([output_len])
y = tf.matmul(x2_drop, W2) + b2

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, output_len])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
# loss = tf.reduce_mean(
#   tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
loss = tf.losses.mean_squared_error(labels=y_, predictions=y)
accuracy = tf.reduce_mean(tf.squared_difference(y, y_))
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
# train_step = tf.train.AdamOptimizer(1).minimize(loss)


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for i in range(10000):
    batch_xs, batch_ys = resample(x_train, y_train, n_samples=100, random_state=0)
#     sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    

In [ ]:
print("test accuracy %g"%accuracy.eval(feed_dict={
x: x_test, y_: y_test}))